# Генеративные состязательные сети

https://github.com/eriklindernoren/Keras-GAN/blob/master/gan/gan.py

В этой тетрадке мы научимся писать сети на чуть более низком уровне.

Обучаем две сети: одна генерирует данные, другая отличает фейковые от настоящих. Рано или поздно у нас будет генератор данных, который можно будет использовать.

Рассказывать ли WGAN и идеи Арьевского или всё ещё обычный? По-моему уже пора.

# Применения

Ганы стали применять очень много где.

Если после 2012 года и работы ЛеКуна было много хайпа, то после 2014 его стало совсем много, как раз из-за ганов.

Вот наиболее значимые из них:

- **CycleGAN**.
- **WGAN**.
- **Unsupervised NMT**.

In [13]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, LeakyReLU
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model

import numpy as np
from mnist import X
X = X.reshape(-1, 28, 28)

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
latent_dim = 64
batch_size = 64
sample_interval = 50
epochs = 1000

Генератор сделаю полносвязным. Для желающих: напишите через deconvolution-ы.

In [ ]:
G = Sequential([
    Dense(256, input_dim=latent_dim),
    LeakyReLU(),
    BatchNormalization(),
    Dense(512),
    LeakyReLU(),
    BatchNormalization(),
    Dense(784),
    LeakyReLU(),
    BatchNormalization(),
    Reshape((28, 28))
])

#G.compile()
G.summary()

In [ ]:
D = Sequential([
    Dense(512, input_dim=784),
    LeakyReLU(),
    Dense(256),
    LeakyReLU(),
    Dense(64),
    LeakyReLU(),
    Dense(1, activation='sigmoid')
])

D.compile(loss='binary_crossentropy', optimizer='adam')
D.summary()

Теперь сольем их в одну модель.

In [ ]:
z = Input(shape=(latent_dim,))
img = G(z)
validity = D(img)
GAN = Model(z, validity)
GAN.compile(loss='binary_crossentropy', optimizer='adam')

Самое содержательное: цикл обучения. Его нужно написать вручную — model.fit() не прокатит.

In [ ]:
# эпохи — это в смысле батчи
for epoch in range(epochs):
    real = X[np.random.randint(0, X.shape[0], batch_size)] # так можно посэмплить батч
    noise = np.random.randn((batch_size, latent_dim))
    fake = G.predict(noise)
    
    d_loss_real = D.train_on_batch(real, np.ones((batch_size, 1)))
    d_loss_fake = D.train_on_batch(fake, np.zeros((batch_size, 1)))
    d_loss = np.add(d_loss_real, d_loss_fake)

    noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

    g_loss = GAN.train_on_batch(noise, valid)

    print ("%d, D loss: %f, G loss: %f" % (epoch, d_loss[0], g_loss))

    if epoch % sample_interval == 0:
        # выводим то, что в батче
        # дополнительных вычислений это не стоит
        fake = fake.reshape(-1, 28, 28)
        
        fig, axs = plt.subplots(8, 8)
        for i in range(8):
            for j in range(8):
                axs[i,j].imshow(gen_imgs[i*8 + j, :,:, 0], cmap='gray')
                axs[i,j].axis('off')
        fig.savefig("images/%d.png" % epoch)
        plt.close()

# Вассерштейново расстояние и эвристики

Обучение GAN-ов не на игрушечных данных **очень** нестабильно. Придумали множество костылей, чтомы его улучшить:
* Делать лэйблы не 0 и 1, а 0.1 и 0.9 или что-то близкое.
* Выкинуть нафиг эту KL-дивергенцию. Оказывается, она не сходится и может вызывать взрывающие / затухающие градиенты. Вместо неё можно использовать Earth Moving.
* Напрямую делать penalty на слишком большой или слишком маленький градиент.
* На каждую итерацию итератора несколько итераций дискриминатора.